In [4]:
import open3d
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import copy

import utils.pointcloud as pointcloud
import utils.registration as registration
import utils.grid_search_rs_unopt as grid_search

from utils.depth_camera import DepthCamera, DepthCameraParams

In [2]:
local_data = np.load("temp/trajectory/local.npz", allow_pickle=True)

In [3]:
sequence_ts = local_data["sequence_ts"]
local_t = local_data["local_t"]
local_pcds = local_data["local_pcds"]
num_frames = len(sequence_ts) - 1

local_pcds = [pointcloud.make_pcd(local_pcds[i]) for i in range(num_frames)]

In [4]:
num_frames, len(local_pcds), len(local_t)

(599, 599, 599)

In [ ]:
for i in range(1, num_frames, 20):
    registration.view(local_pcds[i], local_pcds[i - 1], local_t[i])

In [5]:
trajectory_t = [np.identity(4)]

for t in range(1, num_frames):
    trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))
    
trajectory_pcd = []

for i in range(num_frames):
    pcd = copy.deepcopy(local_pcds[i])
    pcd.transform(trajectory_t[i])
    trajectory_pcd.append(pcd)

trajectory_pcd = pointcloud.merge_pcds(trajectory_pcd, 0.05)

In [6]:
pointcloud.view(trajectory_pcd)

In [9]:
global_files = glob.glob("temp/trajectory/global_*.npz")
global_ts = [int(os.path.basename(f).split(".")[0].split("_")[1]) for f in global_files]

In [10]:
global_inds = []

for i in range(len(global_ts)):
    index = np.argwhere(sequence_ts == global_ts[i])[0][0]
    global_inds.append(index)
    # print(f"Global {global_ts[i]} | Local {sequence_ts[index]} | Index {index}")

In [11]:
global_target_t = []

for i in range(len(global_ts)):
    global_data = np.load(f"temp/trajectory/global_{global_ts[i]}.npz", allow_pickle=True)
    
    source = global_data["source"]
    target = global_data["target"]
    
    source_feat = global_data["source_feat"]
    target_feat = global_data["target_feat"]
    
    source, target, result = grid_search.global_registration(source, source_feat, target, target_feat, cell_size=2, n_random=0.5, refine_enabled=True)
    registration.describe(source, target, result)
    
    global_target_t.append(result.transformation)

0
Keypts: [6074, 28716]	No of matches: 1664	Fitness: 0.2740	Inlier RMSE: 0.0283
1
Keypts: [5997, 28822]	No of matches: 2053	Fitness: 0.3423	Inlier RMSE: 0.0279
2
Keypts: [5923, 29492]	No of matches: 2147	Fitness: 0.3625	Inlier RMSE: 0.0278
3
Keypts: [5557, 28699]	No of matches: 1848	Fitness: 0.3326	Inlier RMSE: 0.0273
4
Keypts: [5732, 29894]	No of matches: 2470	Fitness: 0.4309	Inlier RMSE: 0.0273
5
Keypts: [5547, 29316]	No of matches: 2483	Fitness: 0.4476	Inlier RMSE: 0.0278
6
Keypts: [4949, 28902]	No of matches: 3141	Fitness: 0.6347	Inlier RMSE: 0.0274
7
Keypts: [4359, 28716]	No of matches: 2944	Fitness: 0.6754	Inlier RMSE: 0.0273
8
Keypts: [3818, 28399]	No of matches: 2479	Fitness: 0.6493	Inlier RMSE: 0.0277
9
Keypts: [3287, 29985]	No of matches: 2569	Fitness: 0.7816	Inlier RMSE: 0.0276
10
Keypts: [3331, 28240]	No of matches: 2890	Fitness: 0.8676	Inlier RMSE: 0.0268
11
Keypts: [4071, 28595]	No of matches: 3058	Fitness: 0.7512	Inlier RMSE: 0.0277
12
Keypts: [4152, 28937]	No of matches

In [92]:
import utils.transform as transform

def validate(T1, T2, T3, t1, t2, max_dist, max_rot):
    c1 = transform.check(T3, np.dot(T2, t2), max_t=max_dist, max_r=max_rot)
    c2 = transform.check(T3, np.dot(np.dot(T1, t1), t2), max_t=max_dist, max_r=max_rot)
    c3 = transform.check(T2, np.dot(T1, t1), max_t=max_dist, max_r=max_rot)

    print(f"Check 1: {c1}, Check 2: {c2}, Check 3: {c3}")
    
    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 2:
        raise Exception("Invalid combination")

    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 0:
        raise Exception("Invalid transformations")

    # If all the checks are valid, there is no need of correction
    if c1 and c2 and c3:
        print(":: No need of correction.")
        return T1, T2, T3
    
    # If two checks are wrong, only one transformation needs correction
    if c1:
        # print(":: Correcting Previous Transformation")
        T1 = np.dot(T2, transform.inv_transform(t1))
    elif c2:
        # print(":: Correcting Current Transformation")
        T2 = np.dot(T1, t1)
    else:
        # print(":: Correcting Future Transformation")
        T3 = np.dot(T2, t2)

    return T1, T2, T3


def merge_transformation_matrices(start_t, end_t, local_t):
    local_ts = np.identity(4)

    for t in range(start_t, end_t):
        local_ts = np.dot(local_t[t + 1], local_ts)
        
    return local_ts

In [15]:
global_t = [np.identity(4) for _ in range(num_frames)]

found_correct_global = False
found_correct_global_at = -1

for t in range(len(global_inds)):
    if t > 1 and not found_correct_global:
        print(f"Global registration verification: {t}/{len(global_inds)}")
        total = 0
        for i in range(t, t - 3, -1):
            if np.sum(global_target_t[i]) == 4:
                total += 1
                
        print(f"Total invalid global registrations: {total}")        
        if total > 1: continue
        
        print(f"Validating and correcting global registrations.")
        try:
            global_target_t[t - 2], global_target_t[t - 1], global_target_t[t] = validate(
                global_target_t[t - 2], global_target_t[t - 1], global_target_t[t], 
                merge_transformation_matrices(global_inds[t - 2], global_inds[t - 1], local_t),
                merge_transformation_matrices(global_inds[t - 1], global_inds[t], local_t),
                max_rot=2, max_dist=0.1
            )
            found_correct_global = True
            found_correct_global_at = t
        except Exception as e:
            print(e)
            continue

Global registration verification: 2/25
Total invalid global registrations: 0
Validating and correcting global registrations.
Check 1: True, Check 2: True, Check 3: True
:: No need of correction.


In [16]:
if found_correct_global:
    global_t[global_inds[found_correct_global_at]] = global_target_t[found_correct_global_at]

    for t in range(global_inds[found_correct_global_at] + 1, num_frames - 1):
        global_t[t] = np.dot(global_t[t - 1], local_t[t])
        
    for t in range(global_inds[found_correct_global_at] - 1, -1, -1):
        global_t[t] = np.dot(global_t[t + 1], transform.inv_transform(local_t[t + 1]))

In [19]:
for i in range(0, num_frames, 50):
    registration.view(local_pcds[i], target, global_t[i])

In [ ]:
trajectory_pcd = []

for i in range(num_frames):
    pcd = copy.deepcopy(local_pcds[i])
    pcd.transform(global_t[i])
    trajectory_pcd.append(pcd)

trajectory_pcd = pointcloud.merge_pcds(trajectory_pcd, 0.05)

In [44]:
class GlobalRegistrationVerification:
    
    def __init__(self):
        self.local_t = []
        self.global_t = []
        self.sequence_ts = []
        self.global_inds = []
        self.global_pcds = []
        self.local_pcds = []
        self.global_target_t = []
        self.found_correct_global = False
        self.found_correct_global_at = -1
        
    def update_local(self, local_timestamp, local_pcd, local_transformation):
        self.sequence_ts.append(local_timestamp)
        self.local_t.append(local_transformation)
        self.local_pcds.append(local_pcd)
        
        if self.found_correct_global:
            self.global_t.append(np.dot(self.global_t[-1], local_transformation))
        else:
            self.global_t.append(np.identity(4))
        
    
    def update_global(self, global_timestmap, global_pcd, global_transformation):
        index = np.argwhere(np.array(self.sequence_ts) == global_timestmap).flatten()
        print(index)
        
        if len(index) == 0:
            print(f"Timestamp {global_timestmap} not found in sequence.")
        else:
            index = index[0]
        
        self.global_inds.append(index)
        self.global_target_t.append(global_transformation)
        self.global_pcds.append(global_pcd)
        
        if len(self.global_inds) > 2 and not self.found_correct_global:
            self.verify()
            
    
    def verify(self):
        for t in range(len(self.global_inds)):
            if t > 1:
                print(f"Global registration verification: {t}/{len(self.global_inds)}")
                total = 0
                for i in range(t, t - 3, -1):
                    if np.sum(self.global_target_t[i]) == 4:
                        total += 1
                        
                print(f"Total invalid global registrations: {total}")        
                if total > 1: return
                
                print(f"Validating and correcting global registrations.")
                try:
                    self.global_target_t[t - 2], self.global_target_t[t - 1], self.global_target_t[t] = validate(
                        self.global_target_t[t - 2], self.global_target_t[t - 1], self.global_target_t[t], 
                        merge_transformation_matrices(self.global_inds[t - 2], self.global_inds[t - 1], self.local_t),
                        merge_transformation_matrices(self.global_inds[t - 1], self.global_inds[t], self.local_t),
                        max_rot=2, max_dist=0.1
                    )
                    self.found_correct_global = True
                    self.found_correct_global_at = t
                except Exception as e:
                    print(f"Exception:", e)
                    return
        
        if self.found_correct_global:
            self.global_t[self.global_inds[self.found_correct_global_at]] = self.global_target_t[self.found_correct_global_at]

            for t in range(self.global_inds[self.found_correct_global_at] + 1, len(self.global_t)):
                self.global_t[t] = np.dot(self.global_t[t - 1], self.local_t[t])
                
            for t in range(self.global_inds[self.found_correct_global_at] - 1, -1, -1):
                self.global_t[t] = np.dot(self.global_t[t + 1], transform.inv_transform(self.local_t[t + 1]))

In [45]:
grv = GlobalRegistrationVerification()

In [46]:
for i in range(num_frames):
    grv.update_local(sequence_ts[i], local_pcds[i], local_t[i])

In [47]:
for i in range(len(global_ts)):
    grv.update_global(global_ts[i], target, global_target_t[i])

[0]
[25]
[50]
Global registration verification: 2/3
Total invalid global registrations: 0
Validating and correcting global registrations.
Check 1: True, Check 2: True, Check 3: True
:: No need of correction.
[75]
[100]
[125]
[150]
[175]
[200]
[225]
[249]
[274]
[299]
[324]
[349]
[374]
[399]
[424]
[449]
[474]
[499]
[524]
[549]
[574]
[]
Timestamp 1685247662460 not found in sequence.


In [51]:
for i in range(0, num_frames, 50):
    registration.view(grv.local_pcds[i], grv.global_pcds[0], grv.global_t[i])

In [59]:
depth_files = glob.glob("temp/hololens/data_*.npz")

sequence_ts = np.array([int(os.path.basename(f).split("_")[1].split(".")[0]) for f in depth_files])

In [30]:
t = sequence_ts[10]

depth_file = f"temp/hololens/data_{t}.npz"

depth_data = np.load(depth_file)

depth = depth_data["depth"]
pose = depth_data["pose"]

fx: 174.93409729003906, fy: 179.96331787109375, cx: 168.45681762695312, cy: 171.1561737060547
width: 320, height: 288

In [31]:
camera_params = DepthCameraParams("metadata/hololens.json")
depth_camera = DepthCamera(camera_params)

In [55]:
# pcd = depth_camera.depth_image_to_point_cloud(depth)
pcd = pointcloud.make_pcd(depth.reshape(-1, 3))

In [157]:
local_pcds = []
global_t = []
local_t = [np.identity(4)]

for t in sequence_ts[:5]:
    depth_file = f"temp/hololens/data_{t}.npz"

    depth_data = np.load(depth_file)

    depth = depth_data["depth"]
    pose = depth_data["pose"].T
    
    pcd = pointcloud.make_pcd(depth.reshape(-1, 3))
    # pcd.transform(pose)
    
    local_pcds.append(pcd)
    global_t.append(pose)
    
    if len(local_pcds) > 1:
        t = np.dot(transform.inv_transform(global_t[-2]), global_t[-1])
        local_t.append(t)
    
    
# local_pcds = pointcloud.merge_pcds(local_pcds, 0.05)

In [158]:
for i in range(1, 5):
    print(i, i - 1)
    t = np.dot(transform.inv_transform(global_t[i - 1]), global_t[i])
    registration.view(local_pcds[i], local_pcds[i - 1], t)

1 0
2 1
3 2
4 3


In [132]:
for i in range(len(local_pcds)):
    local_pcds[i].transform(local_t[i])

In [133]:
open3d.visualization.draw_geometries(local_pcds)

In [161]:
num_frames = len(local_pcds)

trajectory_t = [np.identity(4)]

for t in range(1, num_frames):
    trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))
    
trajectory_pcd = []

for i in range(num_frames):
    pcd = copy.deepcopy(local_pcds[i])
    pcd.transform(trajectory_t[i])
    trajectory_pcd.append(pcd)

trajectory_pcd = pointcloud.merge_pcds(trajectory_pcd, 0.05)

open3d.visualization.draw_geometries([trajectory_pcd])